# Hardpoint breakaway tests recent history

In [ ]:
# Times Square parameters

days_to_plot = 50


In [ ]:
import asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
def stiffness(force, disp, name):
    forces = force.values
    disps = disp.values
    if name == 'Pos':
        maxi = np.argmax(force.values > 1000.0)
        mini = np.argmin(force.values < -1000.0)
    elif name == 'Neg':
        maxi = np.argmin(force.values > 1000.0)
        mini = np.argmax(force.values < -1000.0)
    forceMax = forces[maxi] 
    forceMin = forces[mini]
    dispMax = disps[maxi] * 1.0E6
    dispMin = disps[mini] * 1.0E6
    stiffness = (forceMax - forceMin) / (dispMax - dispMin) 
    breakaway_plus = np.max(forces)
    breakaway_minus = np.min(forces)
    return breakaway_plus, breakaway_minus, stiffness


In [ ]:
end = Time.now()
start = end - TimeDelta(days_to_plot, format='jd')
client = EfdClient('usdf_efd')

hardpoints = await client.select_time_series(\
                    "lsst.sal.MTM1M3.logevent_hardpointTestStatus", \
                    ["*"], start, end)
len(hardpoints)

In [ ]:
%matplotlib inline

states = [['Pos',3,4], ['Neg',4,5]]

fig, axs = plt.subplots(1,2,figsize=(10,5))
fig.suptitle("MTM1M3 hardpoint trending", fontsize=18)
axs[0].set_title("Breakaway forces", fontsize=14)
axs[0].set_ylim(-4000, 4000)
axs[0].set_ylabel("Breakaway force (N)")
axs[1].set_title("Stiffness", fontsize=14)
axs[1].set_ylim(0, 50)
axs[1].set_ylabel("Stiffness (N/micron)")
min_time = 1.0E12
max_time = 0.0

queries = []
qTimes = []
starts = []

for [name, startState, endState] in states:
    for index in range(6):
        stiffs = []
        pos_breakaway = []
        neg_breakaway = []
        times = []
        getStarts = hardpoints[hardpoints[f'testState{index}']==startState]['private_kafkaStamp']
        getEnds = hardpoints[hardpoints[f'testState{index}']==endState]['private_kafkaStamp']
        print(len(getStarts), len(getEnds))
        for n in range(len(getStarts)):
            start = Time(getStarts.iloc[n], format='unix_tai', scale='tai')
            end = start + TimeDelta(480, format='sec') #Time(getEnds.iloc[n], format='unix_tai', scale='tai')
            try:
                t0 = Time.now()
                hardpointData = await client.select_time_series(\
                    "lsst.sal.MTM1M3.hardpointActuatorData", \
                    [f'displacement{index}', f'measuredForce{index}'], start.utc, end.utc)
                t1 = Time.now()

                deltaT = (t1 - t0).value * 86400 * 1000
                query = f"lsst.sal.MTM1M3.hardpointActuatorData, displacement{index}, measuredForce{index}, {start.utc.isot}, {end.utc.isot}"
                queries.append(query)
                qTimes.append(deltaT)
                starts.append(start)
                print(index, n, f"Query time = {(end.value - start.value):.2f} seconds.  Query took {deltaT:.1f} msec. {start.utc.isot}")
                
                force = hardpointData[f'measuredForce{index}']
                disp = hardpointData[f'displacement{index}']
                forceMin, forceMax, stiff = stiffness(force, disp, name)
                pos_breakaway.append(forceMax)
                neg_breakaway.append(forceMin)
                stiffs.append(stiff)
                time = start.utc.unix_tai
                times.append(time)
                if time < min_time:
                    min_time = time
                    min_time_list = start.isot.split('.')[0]
                if time > max_time:
                    max_time = time
                    max_time_list = start.isot.split('.')[0]
            except:
                continue

        if name == 'Pos':
            axs[0].plot(times, pos_breakaway, marker='x', label=f"{index}")
            axs[0].plot(times, neg_breakaway, marker='x', label=f"{index}")
        elif name == 'Neg':
            axs[0].plot(times, pos_breakaway, marker='x', label='')
            axs[0].plot(times, neg_breakaway, marker='x', label='')
        axs[1].plot(times, stiffs, marker='x', label=f"{name}_{index}")
        axs[0].set_xticks([min_time, max_time],[min_time_list, max_time_list], rotation=10)
        axs[1].set_xticks([min_time, max_time],[min_time_list, max_time_list], rotation=10)

axs[0].legend(loc='center left', ncol=3)
axs[1].legend(loc='upper left', ncol=3)
#plt.show()
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Trends_18Oct24.pdf")

In [ ]:
bins = np.linspace(0,60000, 50)
plt.hist(qTimes, bins=bins)
plt.title("Histogram of MTM1M3 hardpoint query times")
longQuery = 10.0 * np.min(qTimes)
long = [q for q in qTimes if q > longQuery]
pct = len(long) / len(qTimes) * 100.0
plt.text(5000, 70, f"{pct:.1f} percent are longer than 10 times the minimum time")
plt.xlabel("EFD query times at USDF (ms)")
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Query Times_18Oct24.png")

In [ ]:
file = open("/home/c/cslage/u/MTM1M3/data/Hardpoint_Query Times_18Oct24.txt", 'w')
for i in range(len(qTimes)):
    file.write(f"{qTimes[i]:.2f}: \t {queries[i]} \n")
file.close()

In [ ]:
print(len(starts), len(qTimes))
print(starts[0].isot, qTimes[0])
print(type(starts[0]))

In [ ]:
fig = plt.figure(figsize = (12,5))
ax = plt.axes([0.10,0.30,0.80,0.60])
plotStarts = []
for start in starts:
    plotStarts.append(start.unix_tai)
tick = Time("2024-09-05T00:00", format='isot')
xticklabels = []
xticks = []
for i in range(45):
    xticks.append(tick.unix_tai)
    xticklabels.append(tick.isot.split("T")[0])
    tick += TimeDelta(1, format='jd')
ax.scatter(plotStarts, qTimes)
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, rotation=90)
ax.set_ylabel("EFD query time (msec)")
ax.set_title("MTM1M3 hardpoint query times vs date")
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Query_Time_Trend_19Oct24.png")

In [ ]:
plotStarts[-1]

In [ ]:
tick.isot.split("T")[0]